In [5]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

import torch
from models.resnet import ResNet

BLOCKS = [50, 101, 152][0]

CKP_PATH = f'loaded_models/resnet{BLOCKS}.pth'
SAVE_PATH = f'../weights/imagenet_resnet{BLOCKS}.pth'

resnet = ResNet('classic', BLOCKS)
resnet_ckp = torch.load(CKP_PATH, weights_only=False)

In [ ]:
i = 0
for key, value in resnet.state_dict().items():
    print(f"{i}: {key}: {value.shape}")
    i += 1
print('------------------------------------------------------------------')
i = 0
for key, value in resnet_ckp.items():
    print(f"{i}: {key}: {value.shape}")
    i += 1

In [6]:
PATH_TO_MAPPING = f'mappings/resnet{BLOCKS}_weights_mapping.txt'

mapping = {}
with open(PATH_TO_MAPPING, 'r') as f:
    for line in f.readlines():
        cline = line[:-1]
        current_layer = int(cline.split(':')[0])
        
        if ('stem.stem.0.bias' not in cline) and ('num_batches_tracked' not in cline):
            old_layer = int(cline.split('(')[-1].split(')')[0])
        else:
            old_layer = None
            
        mapping[current_layer] = old_layer

i = 0
current_state = {}
for key, value in resnet.state_dict().items():
    current_state[i] = (key, value)
    i += 1
    
i = 0
old_state = {}
for key, value in resnet_ckp.items():
    old_state[i] = value
    i += 1
    
new_state = {}
for i in range(len(current_state)):
    old_i = mapping[i]
    
    if old_i is None:
        value = current_state[i][1]
    else:
        value = old_state[old_i]
        
    new_state[current_state[i][0]] = value

In [7]:
resnet.load_state_dict(new_state)
torch.save(resnet.state_dict(), SAVE_PATH)